# **AutoGen AI Agents for VO2Max**

## **VO₂ Max** (maximal oxygen uptake) is the maximum rate at which an individual can consume oxygen during intense or maximal exercise. It’s a key indicator of **cardiovascular fitness** and **aerobic endurance**.

## _VO₂ Max = Q×(CaO₂−CvO₂) / Body Weight (kg)_

### Setup

In [2]:
import os
from autogen import ConversableAgent
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-4o-mini"}

### Define an AutoGen agent to get athlete info (**Agent A**)

In [3]:
athlete_info_agent = ConversableAgent(
    name="AthleteInformationAgent",
    system_message='''You are a helpful athlete support agent,
    you are here to help athletes determine their health from VO2Max readings.
    Your job is to gather the athlete's name, sex, and age.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

### Define an AutoGen agent to get the athlete's VO2Max value (**Agent B**)

In [4]:
athlete_vo2max_agent = ConversableAgent(
    name="AthleteVO2MaxAgent",
    system_message='''You are a helpful athlete analyst agent,
    you are here to help athletes determine their health from VO2Max readings.
    Your job is to gather the athlete's VO2Max rating and analyze this VO2Max value against their sex and age providing a summary of their health.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

### Define an AutoGen agent to act as the athlete's proxy (**Agent P**)

In [5]:
athlete_proxy_agent = ConversableAgent(
    name="AthleteProxyAgent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "TERMINATE" in msg.get("content").lower(),
)

### Craft a series of tasks to analyze the athlete's VO2MAX

In [6]:
tasks = [
    {
        "sender": athlete_info_agent,
        "recipient": athlete_proxy_agent,
        "message": 
            "Hello, I'm here to help you get started with your VO2Max analysis."
            "Could you please tell me your name, sex, and age?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'sex': '', 'age': ''}",
        },
        "max_turns": 3,
        "clear_history" : True
    },
    {
        "sender": athlete_vo2max_agent,
        "recipient": athlete_proxy_agent,
        "message": 
                "Great! "
                "What is your VO2Max value?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": athlete_proxy_agent,
        "recipient": athlete_vo2max_agent,
        "message": "Let's analyze your VO2Max and summarize your health.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

### Start the process of collecting and analyzing VO2Max results

In [ ]:
from autogen import initiate_chats

task_results = initiate_chats(tasks)

for task_result in task_results:
    print(task_result.summary)
    print("\n")

### How much did that just cost?

In [ ]:
for task_result in task_results:
    print(task_result.cost)
    print("\n")